## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-01-04-09-09 +0000,missionlocal,Llevando el fuego a casa: Cientos se reúnen en...,https://missionlocal.org/2025/12/llevando-el-f...
1,2026-01-01-04-00-00 +0000,wsj,China Signals It Won’t Give an Inch to the U.S...,https://www.wsj.com/world/americas/china-signa...
2,2026-01-01-04-00-00 +0000,wsj,Russia’s Strikes on Odesa Aim to Cut Ukraine’s...,https://www.wsj.com/world/russia-ukraine-odesa...
3,2026-01-01-04-00-00 +0000,wsj,A Booze Ban Since the ’50s Is Being Quietly Li...,https://www.wsj.com/world/middle-east/saudi-ar...
4,2026-01-01-03-55-03 +0000,startribune,Cities around the world welcome 2026 with thun...,https://www.startribune.com/new-zealands-auckl...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2413/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
72,trump,33
116,year,30
115,new,28
91,guard,12
176,california,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
95,2025-12-31-21-46-14 +0000,nypost,Nasty weather on tap for New Year fetes at Ros...,https://nypost.com/2025/12/31/us-news/nasty-we...,116
14,2026-01-01-02-19-53 +0000,nypost,Trump calls for ‘peace on Earth’ at glittering...,https://nypost.com/2025/12/31/us-news/trump-ca...,108
196,2025-12-31-11-05-20 +0000,nypost,Washington Monument will glow on New Year’s Ev...,https://nypost.com/2025/12/31/us-news/washingt...,89
67,2025-12-31-23-02-41 +0000,nyt,"In New Year’s Speech to Russia, Putin Says Lit...",https://www.nytimes.com/2025/12/31/world/europ...,87
126,2025-12-31-19-02-59 +0000,nypost,NJ drivers hit with 9% state gas tax hike to k...,https://nypost.com/2025/12/31/us-news/nj-drive...,85


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
95,116,2025-12-31-21-46-14 +0000,nypost,Nasty weather on tap for New Year fetes at Ros...,https://nypost.com/2025/12/31/us-news/nasty-we...
101,79,2025-12-31-21-20-43 +0000,nypost,"Trump pulling National Guard from Chicago, Por...",https://nypost.com/2025/12/31/us-news/trump-re...
139,35,2025-12-31-17-56-58 +0000,nypost,OpenAI’s pay tops every major tech startup as ...,https://nypost.com/2025/12/31/business/openais...
127,34,2025-12-31-19-02-13 +0000,nypost,DOJ ‘working around the clock’ on Epstein file...,https://nypost.com/2025/12/31/us-news/doj-work...
176,34,2025-12-31-14-48-14 +0000,nypost,"Russian drone attack injures 6, including chil...",https://nypost.com/2025/12/31/world-news/russi...
7,33,2026-01-01-03-28-50 +0000,cbc,U.S. military says 8 killed in strikes on 5 mo...,https://www.cbc.ca/news/world/us-venezuela-dru...
96,33,2025-12-31-21-41-11 +0000,nypost,MTG calls for a ‘tax revolt’ after accusing Tr...,https://nypost.com/2025/12/31/us-news/marjorie...
42,32,2026-01-01-00-11-00 +0000,wsj,Trump’s surprise veto of a Colorado water proj...,https://www.wsj.com/politics/policy/trump-veto...
193,29,2025-12-31-11-53-55 +0000,nypost,Minnesota day care scandal sparks concern over...,https://nypost.com/2025/12/31/us-news/minnesot...
49,28,2025-12-31-23-37-55 +0000,nypost,Hero grandfather dies after running into house...,https://nypost.com/2025/12/31/us-news/grandfat...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
